<a href="https://colab.research.google.com/github/akshan-main/nanobananaexperiment/blob/main/nanobananarestoration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Run the first 2 cells twice each 1st cell->2nd-->1st-->2nd

In [ ]:
!pip uninstall -y numpy
!pip install --no-cache-dir -q "numpy==2.1.1"
import os; os.kill(os.getpid(), 9)


Found existing installation: numpy 2.1.1
Uninstalling numpy-2.1.1:
  Successfully uninstalled numpy-2.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 85.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.46.0 requires huggingface-hub<1.0,>=0.33.5, but you have huggingface-hub 0.26.2 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.1.1 which is incompatible.


In [80]:
from google.colab import userdata
userdata.get('GOOGLE_API_KEY')

ValueError: Please enter a valid secret name

In [76]:
# --- Install deps (no heavy local models) ---
!pip install --no-cache-dir -q \
  "pandas==2.2.2" \
  "pyarrow==16.1.0" \
  "pillow==10.3.0" \
  "scikit-image==0.24.0" \
  "huggingface_hub==0.26.2" \
  "transformers==4.45.2" \
  "diffusers==0.31.0" \
  "accelerate==1.0.1" \
  "datasets==3.0.1" \
  "lpips==0.1.4"

import numpy as np
import pandas as pd
import pyarrow
import PIL
import skimage
import huggingface_hub
import transformers
import diffusers
import accelerate
import datasets
import lpips

print("datasets:", datasets.__version__)


!pip install google-genai

from google.colab import drive
drive.mount('/content/drive')

import os, gc, io, torch, numpy as np, pandas as pd
from pathlib import Path
from PIL import Image

GOOGLE_API_KEY = """enter key here"""
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

PROJECT_ROOT = Path("/content")
INPUT_DIR    = PROJECT_ROOT / "restoration_inputs"
OUT_ROOT     = Path("/content/drive/MyDrive/restoration_results_gemini")
OUT_IMG      = OUT_ROOT / "gen"
OUT_ROOT.mkdir(parents=True, exist_ok=True)
OUT_IMG.mkdir(parents=True, exist_ok=True)

SEED = 123
import random
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

N_IMAGES = 5

MAX_SIDE = 512

print("Drive mounted. Outputs ->", OUT_ROOT)


datasets: 3.0.1
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted. Outputs -> /content/drive/MyDrive/restoration_results_gemini


In [77]:
from google import genai
from google.genai import types
from io import BytesIO

api_key = os.environ.get("GOOGLE_API_KEY")
assert api_key and api_key != "PASTE_YOUR_KEY_HERE", "Set GOOGLE_API_KEY in Cell 1."
client = genai.Client(api_key=api_key)

def _snap(x, m=8):
    return int(max(m, (x // m) * m))

def to_rgb_opaque(img, bg=(255,255,255)):
    if img.mode in ("RGBA","LA") or (img.mode=="P" and "transparency" in img.info):
        base = Image.new("RGB", img.size, bg)
        if img.mode != "RGBA": img = img.convert("RGBA")
        base.paste(img, mask=img.split()[-1])
        return base
    return img.convert("RGB")

def prep(img: Image.Image, max_side=MAX_SIDE) -> Image.Image:
    img = to_rgb_opaque(img)
    w,h = img.size
    s = min(1.0, max_side / max(w,h))
    img = img.resize((_snap(int(w*s)), _snap(int(h*s))), Image.BICUBIC)
    return img

import time
from io import BytesIO

GEMINI_MODEL = "gemini-2.5-flash-image-preview"

def _extract_image(resp):
    if not getattr(resp, "candidates", None):
        return None
    for cand in (resp.candidates or []):
        content = getattr(cand, "content", None)
        if not content:
            continue
        for part in (getattr(content, "parts", None) or []):
            inline = getattr(part, "inline_data", None)
            if inline is not None and getattr(inline, "data", None):
                try:
                    return Image.open(BytesIO(inline.data)).convert("RGB")
                except Exception:
                    pass
    return None

def gemini_restore(damaged_pil: Image.Image, prompt: str,
                   max_retries: int = 10, backoff_s: float = 1.0) -> Image.Image:
    """
    Calls Gemini with image-only response. If it fails to return an image,
    retries and also tries a slightly smaller version of the input image.
    """
    base = to_rgb_opaque(damaged_pil)

    w, h = base.size
    w2 = max(8, (w // 8) * 8 - 16)
    h2 = max(8, (h // 8) * 8 - 16)
    smaller = base if (w2 == w and h2 == h) else base.resize((w2, h2), Image.BICUBIC)

    images_to_try = [base, smaller]
    prompts_to_try = [
        prompt.strip() + " Return an image only. Do not include any text.",
        prompt.strip(),
    ]

    last_err = None
    for attempt in range(1, max_retries + 1):
        for pr in prompts_to_try:
            for img_in in images_to_try:
                try:
                    resp = client.models.generate_content(
                        model=GEMINI_MODEL,
                        contents=[pr, img_in],
                        config=types.GenerateContentConfig(response_modalities=["IMAGE"])
                    )
                    img = _extract_image(resp)
                    if img is not None:
                        return img
                except Exception as e:
                    if "404" in str(e) or "NOT_FOUND" in str(e):
                        raise RuntimeError(f"Model not found: {GEMINI_MODEL}") from e
                    last_err = e
        time.sleep(backoff_s * attempt)

    raise RuntimeError(f"No image after {max_retries} attempts: {last_err}")


In [74]:
import os, glob
from pathlib import Path
from PIL import Image

if 'INPUT_DIR' not in globals():
    INPUT_DIR = Path("/content/restoration_inputs")
if 'N_IMAGES' not in globals():
    N_IMAGES = 5
if 'MAX_SIDE' not in globals():
    MAX_SIDE = 512
INPUT_DIR.mkdir(parents=True, exist_ok=True)

if '_snap' not in globals():
    def _snap(x, m=8): return int(max(m, (x // m) * m))
if 'save_snapped_rgb' not in globals():
    def save_snapped_rgb(pil_img: Image.Image, path: Path, max_side=512):
        im = pil_img.convert("RGB")
        w, h = im.size
        s = min(1.0, max_side / max(w, h))
        im = im.resize((_snap(int(w*s)), _snap(int(h*s))), Image.BICUBIC)
        im.save(path, quality=95)

ZIP_URL = "http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip"
ZIP_PATH = Path("/content/DIV2K_valid_HR.zip")
TARGET_DIR = Path("/content/DIV2K_valid_HR")

if not ZIP_PATH.exists():
    import urllib.request
    print("Downloading DIV2K_valid_HR.zip ...")
    urllib.request.urlretrieve(ZIP_URL, ZIP_PATH)
else:
    print("Zip already present:", ZIP_PATH)

if not TARGET_DIR.exists():
    import zipfile
    print("Extracting ...")
    with zipfile.ZipFile(ZIP_PATH, 'r') as zf:
        zf.extractall("/content/")
else:
    print("Already extracted:", TARGET_DIR)

files = sorted(glob.glob(str(TARGET_DIR / "*.png")))
if len(files) == 0:
    raise RuntimeError("No PNGs found in the extracted DIV2K_valid_HR folder.")

saved = 0
for i, p in enumerate(files):
    if saved >= N_IMAGES:
        break
    img = Image.open(p).convert("RGB")
    out_path = INPUT_DIR / f"div2k_{i:04d}.png"
    save_snapped_rgb(img, out_path, max_side=MAX_SIDE)
    saved += 1
    print("Saved", out_path.name)

print(f"[Official ZIP route] Saved {saved} images into {INPUT_DIR}")


Zip already present: /content/DIV2K_valid_HR.zip
Already extracted: /content/DIV2K_valid_HR
Saved div2k_0000.png
Saved div2k_0001.png
Saved div2k_0002.png
Saved div2k_0003.png
Saved div2k_0004.png
[Official ZIP route] Saved 5 images into /content/restoration_inputs


In [75]:
import io
import numpy as np
from PIL import ImageDraw, ImageFilter, ImageEnhance
from skimage.util import random_noise

def noise_awgn(img, sigma=25):
    arr = np.array(img).astype(np.float32)
    n = np.random.normal(0, sigma, arr.shape).astype(np.float32)
    return Image.fromarray(np.clip(arr + n, 0, 255).astype(np.uint8))

def noise_poisson_gaussian(img, a=5e-4, b=1e-4):
    x = np.array(img).astype(np.float32)/255.0
    var = np.clip(a*x + b, 1e-8, 1.0)
    y = np.clip(x + np.random.normal(0.0, 1.0, x.shape).astype(np.float32)*np.sqrt(var), 0.0, 1.0)
    return Image.fromarray((y*255).astype(np.uint8))

def noise_speckle(img, sigma=0.10):
    x = np.array(img).astype(np.float32)/255.0
    n = np.random.normal(0, sigma, x.shape).astype(np.float32)
    y = np.clip(x + x*n, 0.0, 1.0)
    return Image.fromarray((y*255).astype(np.uint8))

def noise_salt_pepper(img, amount=0.03, salt_vs_pepper=0.5):
    x = np.array(img).astype(np.float32)/255.0
    y = random_noise(x, mode="s&p", amount=amount, salt_vs_pepper=salt_vs_pepper, clip=True)
    return Image.fromarray((np.clip(y,0,1)*255).astype(np.uint8))

def noise_jpeg_artifacts(img, quality=30):
    buf = io.BytesIO()
    img.save(buf, format="JPEG", quality=int(quality))
    buf.seek(0)
    return Image.open(buf).convert("RGB")

def add_random_scratch(img, count=3, width_range=(2,4)):
    im = img.copy(); draw = ImageDraw.Draw(im); w,h = im.size
    for _ in range(count):
        x1,y1 = np.random.randint(0,w), np.random.randint(0,h)
        x2,y2 = np.random.randint(0,w), np.random.randint(0,h)
        width = np.random.randint(width_range[0], width_range[1]+1)
        color = (255,255,255) if np.random.rand()<0.5 else (0,0,0)
        draw.line((x1,y1,x2,y2), fill=color, width=width)
    return im.filter(ImageFilter.GaussianBlur(radius=0.6))

def add_tear(img):
    im = img.copy(); w,h = im.size
    pts = []; start_x = np.random.randint(int(0.2*w), int(0.8*w))
    for i in range(6):
        px = start_x + np.random.randint(-int(0.08*w), int(0.08*w))
        py = int((i+1)*h/7) + np.random.randint(-int(0.05*h), int(0.05*h))
        pts.append((max(0,min(px,w-1)), max(0,min(py,h-1))))
    mask = Image.new("L",(w,h),0)
    ImageDraw.Draw(mask).polygon(pts, fill=255)
    paper = Image.new("RGB",(w,h),(235,235,235))
    return Image.composite(paper, im, mask)

def add_faded_color(img, sat=0.45, contrast=0.8, tint=(230,220,200), alpha=0.22):
    im = ImageEnhance.Color(img).enhance(sat)
    im = ImageEnhance.Contrast(im).enhance(contrast)
    return Image.blend(im, Image.new("RGB", im.size, tint), alpha)

def add_blur(img, radius=1.8):
    return img.filter(ImageFilter.GaussianBlur(radius=radius))

RESTORE_PROMPTS = {
    "awgn":   "Denoise the image while preserving edges and textures. Keep content unchanged.",
    "pgauss": "Remove sensor noise including shot and read noise. Preserve natural detail.",
    "speckle":"Remove multiplicative speckle noise. Keep surfaces smooth without losing detail.",
    "snp":    "Remove salt and pepper noise. Recover clean lines and boundaries.",
    "jpeg":   "Reduce JPEG compression artifacts and blockiness without oversmoothing.",
    "scratch":"Remove scratches and dust marks. Do not alter shapes or content.",
    "tear":   "Repair torn or missing regions to look natural and match surrounding context.",
    "faded":  "Restore natural colors and correct contrast. Avoid oversaturation.",
    "blur":   "Unblur the image, sharpen and restore fine detail while avoiding halos or artifacts."
}

NOISE_DESCRIBE = {
    "awgn":    lambda im: noise_awgn(im, sigma=10),
    "pgauss":  lambda im: noise_poisson_gaussian(im, a=5e-6, b=2e-6),
    "speckle": lambda im: noise_speckle(im, sigma=0.15),
    "snp":     lambda im: noise_salt_pepper(im, amount=0.05, salt_vs_pepper=0.7),
    "jpeg":    lambda im: noise_jpeg_artifacts(im, quality=20),
    "scratch": lambda im: add_random_scratch(im, count=5, width_range=(2,4)),
    "tear":    lambda im: add_tear(im),
    "faded":   lambda im: add_faded_color(im, sat=0.4, contrast=0.7, alpha=0.25),
    "blur":    lambda im: add_blur(im, radius=1.1),
}


SELECTED = list(NOISE_DESCRIBE.keys())
print("Damages:", SELECTED)


Damages: ['awgn', 'pgauss', 'speckle', 'snp', 'jpeg', 'scratch', 'tear', 'faded', 'blur']


In [69]:
import lpips, torch
import torch.nn.functional as F
from PIL import Image, ImageFilter, ImageOps
from transformers import CLIPModel, CLIPProcessor
from skimage.metrics import peak_signal_noise_ratio as psnr_metric
from skimage.metrics import structural_similarity as ssim_metric
from skimage.color import rgb2lab, deltaE_ciede2000

def match_size(ref: Image.Image, img: Image.Image) -> Image.Image:
    return img if img.size == ref.size else img.resize(ref.size, Image.BICUBIC)

def to_rgb_opaque(img, bg=(255,255,255)):
    if img.mode in ("RGBA","LA") or (img.mode=="P" and "transparency" in img.info):
        base = Image.new("RGB", img.size, bg)
        if img.mode != "RGBA": img = img.convert("RGBA")
        base.paste(img, mask=img.split()[-1])
        return base
    return img.convert("RGB")

def autocrop_uniform_border(img: Image.Image, tol=6):
    """
    Trim near-uniform borders (e.g., padding) that Gemini might add,
    then return the cropped image. If nothing to trim, returns original.
    tol: per-channel absolute tolerance [0..255]
    """
    arr = np.asarray(img.convert("RGB"))
    h, w, _ = arr.shape

    # reference border color as median of outermost rows/cols
    border_cols = np.concatenate([arr[0,:,:], arr[-1,:,:], arr[:,0,:], arr[:,-1,:]], axis=0)
    ref = np.median(border_cols, axis=0)

    # mask pixels within tolerance of ref color
    diff = np.abs(arr - ref[None,None,:]).max(axis=2)
    border_mask = diff <= tol

    # find bounding box of non-border
    coords = np.argwhere(~border_mask)
    if coords.size == 0:
        return img  # whole image looks like border= do nothing
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0) + 1
    if (y0 <= 2 and x0 <= 2 and (h - y1) <= 2 and (w - x1) <= 2):
        return img  # cropping negligible
    return img.crop((x0, y0, x1, y1))

def psnr(a: Image.Image, b: Image.Image) -> float:
    A = np.asarray(a).astype(np.float32)/255.0
    B = np.asarray(b).astype(np.float32)/255.0
    return float(psnr_metric(A, B, data_range=1.0))

def ssim(a: Image.Image, b: Image.Image) -> float:
    A = np.asarray(a).astype(np.float32)/255.0
    B = np.asarray(b).astype(np.float32)/255.0
    try:
        return float(ssim_metric(A, B, channel_axis=2, data_range=1.0))
    except TypeError:
        return float(ssim_metric(A, B, multichannel=True, data_range=1.0))

def sobel_edges_gray(img: Image.Image) -> np.ndarray:
    g = ImageOps.grayscale(img)
    e = g.filter(ImageFilter.FIND_EDGES)
    return np.asarray(e).astype(np.float32) / 255.0

def edge_ssim(a: Image.Image, b: Image.Image) -> float:
    Ea = sobel_edges_gray(a); Eb = sobel_edges_gray(b)
    try:
        return float(ssim_metric(Ea, Eb, data_range=1.0))
    except TypeError:
        return float(ssim_metric(Ea, Eb, multichannel=False, data_range=1.0))

def deltaE00(a: Image.Image, b: Image.Image) -> float:
    A = np.asarray(a.convert("RGB")).astype(np.float32)/255.0
    B = np.asarray(b.convert("RGB")).astype(np.float32)/255.0
    LabA = rgb2lab(A); LabB = rgb2lab(B)
    dE = deltaE_ciede2000(LabA, LabB)
    return float(np.mean(dE))

def hist_intersection(a: Image.Image, b: Image.Image, bins=32) -> float:
    A = np.asarray(a.convert("RGB")).reshape(-1,3)
    B = np.asarray(b.convert("RGB")).reshape(-1,3)
    H, _ = np.histogramdd(A, bins=bins, range=((0,255),(0,255),(0,255)))
    G, _ = np.histogramdd(B, bins=bins, range=((0,255),(0,255),(0,255)))
    H = H / (H.sum() + 1e-8); G = G / (G.sum() + 1e-8)
    return float(np.minimum(H, G).sum())

from PIL import ImageDraw, ImageFont


def save_panel(clean, dirty, restored, path, label_text=None):
    clean = to_rgb_opaque(clean)
    dirty = to_rgb_opaque(dirty)
    restored = to_rgb_opaque(restored)

    trimmed = autocrop_uniform_border(restored, tol=6)
    restored = match_size(clean, trimmed)
    dirty = match_size(clean, dirty)

    w, h = clean.size
    canvas = Image.new("RGB", (w * 3, h), (0, 0, 0))
    canvas.paste(clean, (0, 0))
    canvas.paste(dirty, (w, 0))
    canvas.paste(restored, (w * 2, 0))

    if label_text:
        draw = ImageDraw.Draw(canvas)
        font = ImageFont.load_default()
        bbox = draw.textbbox((0, 0), label_text, font=font)
        tw, th = bbox[2] - bbox[0], bbox[3] - bbox[1]
        pad = 12
        draw.rectangle([0, 0, tw + 2 * pad, th + 2 * pad], fill=(0, 0, 0))
        draw.text((pad, pad), label_text, fill=(255, 255, 255), font=font)

    canvas.save(path, quality=95)



lpips_model = lpips.LPIPS(net='vgg').to("cpu").eval()
clip_model  = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to("cpu").eval()
clip_proc   = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

@torch.no_grad()
def lpips_dist(a: Image.Image, b: Image.Image) -> float:
    A = torch.from_numpy(np.asarray(a).astype(np.float32)/255.0).permute(2,0,1).unsqueeze(0) * 2 - 1
    B = torch.from_numpy(np.asarray(b).astype(np.float32)/255.0).permute(2,0,1).unsqueeze(0) * 2 - 1
    return float(lpips_model(A, B).squeeze().item())

@torch.no_grad()
def clip_cosine(a: Image.Image, b: Image.Image) -> float:
    inputs = clip_proc(images=[a, b], return_tensors="pt")
    feats  = clip_model.get_image_features(**inputs)
    feats  = F.normalize(feats, p=2, dim=-1)
    return float(F.cosine_similarity(feats[0], feats[1], dim=0).item())

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/vgg.pth


In [70]:
labels_map = {
    "awgn": "Additive Gaussian Noise",
    "pgauss": "Poisson-Gaussian Noise",
    "speckle": "Speckle Noise",
    "snp": "Salt & Pepper Noise",
    "jpeg": "JPEG Compression Artifacts",
    "scratch": "Scratches",
    "tear": "Torn Region",
    "faded": "Faded Colors",
    "blur": "Blurred Image"
}


In [71]:
import glob

records = []
img_files = sorted([p for p in glob.glob(str(INPUT_DIR / "*"))
                    if p.lower().endswith((".png",".jpg",".jpeg",".bmp",".webp"))])[:N_IMAGES]
print(f"Processing {len(img_files)} images from {INPUT_DIR}")

for idx, path in enumerate(img_files):
    clean = prep(Image.open(path).convert("RGB"))
    stem  = Path(path).stem

    for key in SELECTED:
        dirty   = NOISE_DESCRIBE[key](clean.copy())
        prompt  = RESTORE_PROMPTS[key]
        noise_label = (labels_map.get(key, key.upper())
                       if 'labels_map' in globals() else key.upper())

        try:
            restored = gemini_restore(dirty, prompt)
            if restored is None:
                raise RuntimeError("gemini_restore returned None (no image)")
        except Exception as e:
            records.append({
                "image": stem, "damage": key, "damage_label": noise_label,
                "psnr": np.nan, "ssim": np.nan, "edge_ssim": np.nan,
                "deltaE00": np.nan, "hist_intersection": np.nan,
                "lpips": np.nan, "clip_cosine": np.nan,
                "panel_path": "",
                "restore_error": str(e)
            })
            print(f"[{idx+1}/{len(img_files)}] {stem} + {noise_label} -> SKIPPED (reason: {e})")
            continue

        restored = match_size(clean, autocrop_uniform_border(restored, tol=6))

        panel_path = OUT_IMG / f"{stem}_{key}_panel.jpg"
        save_panel(clean, dirty, restored, panel_path, label_text=f"Damage: {noise_label}")

        m_psnr = psnr(clean, restored); m_ssim = ssim(clean, restored)
        m_edge = edge_ssim(clean, restored); m_de00 = deltaE00(clean, restored)
        m_hist = hist_intersection(clean, restored)
        m_lpip = lpips_dist(clean, restored); m_clip = clip_cosine(clean, restored)

        records.append({
            "image": stem, "damage": key, "damage_label": noise_label,
            "psnr": m_psnr, "ssim": m_ssim, "edge_ssim": m_edge,
            "deltaE00": m_de00, "hist_intersection": m_hist,
            "lpips": m_lpip, "clip_cosine": m_clip,
            "panel_path": str(panel_path),
            "restore_error": ""
        })

        print(f"[{idx+1}/{len(img_files)}] {stem} + {noise_label} -> {panel_path.name} | "
              f"PSNR {m_psnr:.2f}  SSIM {m_ssim:.3f}  EdgeSSIM {m_edge:.3f}  "
              f"ΔE00 {m_de00:.2f}  HistInt {m_hist:.3f}  LPIPS {m_lpip:.3f}  CLIP {m_clip:.3f}")

    gc.collect()

df = pd.DataFrame(records)
csv_path = OUT_ROOT / "metrics.csv"
df.to_csv(csv_path, index=False)
print("\nSaved metrics:", csv_path)
print("Panels folder:", OUT_IMG)
display(df.head(10))


Processing 5 images from /content/restoration_inputs
[1/5] div2k_0000 + Additive Gaussian Noise -> div2k_0000_awgn_panel.jpg | PSNR 15.36  SSIM 0.124  EdgeSSIM 0.016  ΔE00 11.85  HistInt 0.871  LPIPS 0.377  CLIP 0.969
[1/5] div2k_0000 + Poisson-Gaussian Noise -> div2k_0000_pgauss_panel.jpg | PSNR 15.34  SSIM 0.143  EdgeSSIM 0.037  ΔE00 11.77  HistInt 0.874  LPIPS 0.393  CLIP 0.977
[1/5] div2k_0000 + Speckle Noise -> div2k_0000_speckle_panel.jpg | PSNR 18.52  SSIM 0.399  EdgeSSIM 0.092  ΔE00 9.07  HistInt 0.783  LPIPS 0.358  CLIP 0.930
[1/5] div2k_0000 + Salt & Pepper Noise -> div2k_0000_snp_panel.jpg | PSNR 19.35  SSIM 0.463  EdgeSSIM 0.124  ΔE00 7.94  HistInt 0.705  LPIPS 0.306  CLIP 0.982
[1/5] div2k_0000 + JPEG Compression Artifacts -> div2k_0000_jpeg_panel.jpg | PSNR 14.05  SSIM 0.127  EdgeSSIM 0.034  ΔE00 14.03  HistInt 0.674  LPIPS 0.487  CLIP 0.957
[1/5] div2k_0000 + Scratches -> div2k_0000_scratch_panel.jpg | PSNR 16.78  SSIM 0.246  EdgeSSIM 0.047  ΔE00 9.99  HistInt 0.864  LPI

,image,damage,damage_label,psnr,ssim,edge_ssim,deltaE00,hist_intersection,lpips,clip_cosine,panel_path,restore_error
0,div2k_0000,awgn,Additive Gaussian Noise,15.364768,0.124391,0.016416,11.854523,0.870588,0.377399,0.969258,/content/drive/MyDrive/restoration_results_gem...,
1,div2k_0000,pgauss,Poisson-Gaussian Noise,15.341226,0.143468,0.037360,11.766405,0.874442,0.393173,0.976948,/content/drive/MyDrive/restoration_results_gem...,
2,div2k_0000,speckle,Speckle Noise,18.518482,0.399223,0.091607,9.072612,0.783226,0.357859,0.930345,/content/drive/MyDrive/restoration_results_gem...,
3,div2k_0000,snp,Salt & Pepper Noise,19.351012,0.462649,0.124047,7.943851,0.704787,0.306181,0.982452,/content/drive/MyDrive/restoration_results_gem...,
4,div2k_0000,jpeg,JPEG Compression Artifacts,14.045373,0.126559,0.034313,14.034430,0.674136,0.486561,0.957150,/content/drive/MyDrive/restoration_results_gem...,
5,div2k_0000,scratch,Scratches,16.784300,0.245828,0.047320,9.987782,0.864426,0.362824,0.979424,/content/drive/MyDrive/restoration_results_gem...,
6,div2k_0000,tear,Torn Region,19.440290,0.486267,0.182998,7.315063,0.843593,0.250115,0.981491,/content/drive/MyDrive/restoration_results_gem...,
7,div2k_0000,faded,Faded Colors,15.483912,0.173959,0.036390,13.472646,0.288737,0.432440,0.956637,/content/drive/MyDrive/restoration_results_gem...,
8,div2k_0000,blur,Blurred Image,17.345075,0.246734,0.033364,9.484185,0.827207,0.442942,0.972811,/content/drive/MyDrive/restoration_results_gem...,
9,div2k_0001,awgn,Additive Gaussian Noise,18.870191,0.617393,0.111170,8.329775,0.727934,0.178567,0.986755,/content/drive/MyDrive/restoration_results_gem...,


In [42]:
cols = ["psnr","ssim","edge_ssim","deltaE00","hist_intersection","lpips","clip_cosine"]
display(df.groupby("damage")[cols].mean().round(4))


,psnr,ssim,edge_ssim,deltaE00,hist_intersection,lpips,clip_cosine
damage,,,,,,,
awgn,17.1908,0.3705,0.1405,11.7932,0.5266,0.4261,0.9472
blur,20.5470,0.5532,0.1552,7.2835,0.7689,0.3460,0.9649
faded,15.7490,0.4497,0.2027,13.9920,0.1902,0.3181,0.9410
jpeg,16.4660,0.3356,0.1189,12.6523,0.4857,0.4345,0.9458
pgauss,19.0860,0.4826,0.1088,9.7041,0.4782,0.4061,0.9281
scratch,18.8078,0.4795,0.1026,7.9688,0.7865,0.2664,0.9671
snp,19.7086,0.4980,0.1280,8.2089,0.5758,0.3272,0.9553
speckle,19.4344,0.5459,0.1774,8.2691,0.6495,0.2747,0.9743
tear,16.9963,0.4488,0.1774,9.4689,0.7686,0.2670,0.9689


In [43]:
import glob
from pathlib import Path
from PIL import Image
import datetime

panel_paths = sorted(
    [p for p in glob.glob(str(OUT_IMG / "div2k_*_panel.jpg"))] +
    [p for p in glob.glob(str(OUT_IMG / "div2k_*_panel.png"))] +
    [p for p in glob.glob(str(OUT_IMG / "div2k_*_panel.webp"))] +
    [p for p in glob.glob(str(OUT_IMG / "div2k_*_panel.bmp"))]
)

if not panel_paths:
    raise RuntimeError("No DIV2K panels found. Expected files like 'div2k_xxxx_<noise>_panel.jpg' in OUT_IMG.")

pages = [Image.open(p).convert("RGB") for p in panel_paths]

pdf_path = OUT_ROOT / f"div2k_restoration_panels.pdf"

pages[0].save(pdf_path, save_all=True, append_images=pages[1:], resolution=150, quality=95)
print(f"PDF written to: {pdf_path}")

try:
    from google.colab import files
    files.download(str(pdf_path))
except Exception:
    pass


PDF written to: /content/drive/MyDrive/restoration_results_gemini/div2k_restoration_panels_20250929_042007.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>